In [1]:
data_path = "ft-mistral-7b-instruct-v0.2.jsonl"

# Load the JSONL data from the file
import json
with open(data_path, "r") as f:
    raw_judge_results = [json.loads(line) for line in f]
    
def avg(L):
    return sum(L) / len(L)

In [ ]:
judge_results = dict()
judge_results["sl-cai"] = [[] for i in range(44)]

print(f"Total Model Count: {len(judge_results)}")

for result in raw_judge_results:
    category_id = (result["question_id"] - 1) // 10
    if result["model"] in judge_results.keys():
        judge_results[result["model"]][category_id].append(result["score"])

avg_score_overall = dict()
judge_avg_scores_per_category = dict()
for model in judge_results:
    judge_avg_scores_per_category[model] = []
    for category in range(len(judge_results[model])):
        judge_avg_scores_per_category[model].append(avg(judge_results[model][category]))
    
    avg_score_overall[model] = avg(judge_avg_scores_per_category[model])
    
# Sort the model names by their average score
sorted_models = sorted(avg_score_overall, key=avg_score_overall.get, reverse=False)
for model in sorted_models:
    print(f"{model}: {avg_score_overall[model]:.4f}")

# For each model, plot a heat map of the average scores per category
import plotly.express as px

fig = px.imshow(
    [judge_avg_scores_per_category[model] for model in sorted_models],
    color_continuous_scale='Sunsetdark',
    text_auto=".1f",
)

# We want to show all ticks...
fig.update_xaxes(tickvals=list(range(44)))
fig.update_yaxes(tickvals=list(range(len(judge_results))))

# Set the hover value for every grid cell
fig.update_traces(hovertemplate="Category %{x}<br>Model (average score) %{y}<br>Score %{z:.2f}")

# Set text for every tick
from data.meta_info import category_descriptions_shortest
fig.update_xaxes(ticktext=[f"#{i+1}: {category_descriptions_shortest[i]}" for i in range(44)])
fig.update_yaxes(ticktext=[f"{k} ({avg_score_overall[k]:.2f})" for k in sorted_models])

# Rotate the x-axis labels
fig.update_xaxes(tickangle=60)

# Set figure size
fig.update_layout(
    autosize=False,
    width=1200,
    height=950,
)

# Automatically adjust colorbar height
fig.update_layout(coloraxis_colorbar=dict(
    thicknessmode="pixels", thickness=20,
    lenmode="pixels", len=770,
))

# Set tight margins
# fig.update_layout(margin=dict(l=0, r=0, t=4, b=0))
fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

fig.show()

# save the plot as png
fig.write_image("misc/benchmark-results.png", scale=6)

Total Model Count: 1
sl-cai: 0.2773


ModuleNotFoundError: No module named 'data.sorry_bench'